<a href="https://colab.research.google.com/github/harry-stark/TensorRT-BUILD-Templates/blob/main/TensorRT_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook is a template for building TensorRT OSS components, in a Jupyter environment.

Check for GPU and CUDA toolkit.

In [ ]:
%%shell
nvidia-smi
nvcc --version

Build or Install Cmake>3.12

In [1]:
%%shell
sudo apt install build-essential git
git clone https://github.com/Kitware/CMake/; cd CMake
./bootstrap && make && sudo make install

Streaming output truncated to the last 5000 lines.
-- Checking whether CXX compiler struct stat has st_mtim member
-- Checking whether CXX compiler struct stat has st_mtim member - yes
-- Checking whether CXX compiler struct stat has st_mtimespec member
-- Checking whether CXX compiler struct stat has st_mtimespec member - no
-- Looking for include files sys/types.h, ifaddrs.h
-- Looking for include files sys/types.h, ifaddrs.h - found
-- Checking whether CXX compiler has rlimit64
-- Checking whether CXX compiler has rlimit64 - yes
-- Looking for C++ include execinfo.h
-- Looking for C++ include execinfo.h - found
-- Checking whether backtrace works with this C++ compiler
-- Checking whether backtrace works with this C++ compiler - yes
-- Looking for C++ include dlfcn.h
-- Looking for C++ include dlfcn.h - found
-- Checking whether dladdr works with this C++ compiler
-- Checking whether dladdr works with this C++ compiler - yes
-- Looking for C++ include cxxabi.h
-- Looking for C++ inc

Install TensorRT from nvidia(https://developer.nvidia.com/tensorrt). Nvidia doesn't provide direct download access to Tensorrt . So download it to your drive.

Mount Drive and Install TensorRT

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%shell
sudo dpkg -i "/content/drive/MyDrive/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922_1-1_amd64.deb"
sudo apt-get update
sudo apt-get install tensorrt

Selecting previously unselected package nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922 (1-1) ...
Get:1 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  InRelease
Ign:1 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  InRelease
Get:2 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release [569 B]
Get:2 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release [569 B]
Get:3 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release.gpg [801 B]
Get:3 file:/var/nv-tensorrt-repo-ubuntu2004-cuda11.4-trt8.2.0.6-ea-20210922  Release.gpg [801 B]
Get:4 htt

Clone the TensorRT OSS directory. 

In [ ]:
%%shell
git clone -b master https://github.com/nvidia/TensorRT TensorRT

Build TensorRT OSS components. Take extra care of Path variables like TRT_LIBPATH

In [ ]:
%%shell
cd TensorRT
git submodule update --init --recursive
export TRT_LIBPATH=`pwd`/TensorRT-8.2.0.6
mkdir -p build && cd build
cmake .. -DTRT_LIB_DIR=$TRT_LIBPATH -DTRT_OUT_DIR=`pwd`/out
make -j$(nproc)

Cloning into 'TensorRT'...
remote: Enumerating objects: 7169, done.
remote: Counting objects: 100% (2735/2735), done.
remote: Compressing objects: 100% (1419/1419), done.
remote: Total 7169 (delta 1390), reused 2270 (delta 1267), pack-reused 4434
Receiving objects: 100% (7169/7169), 21.15 MiB | 26.09 MiB/s, done.
Resolving deltas: 100% (4154/4154), done.
Submodule 'parsers/onnx' (https://github.com/onnx/onnx-tensorrt.git) registered for path 'parsers/onnx'
Submodule 'third_party/cub' (https://github.com/NVlabs/cub.git) registered for path 'third_party/cub'
Submodule 'third_party/protobuf' (https://github.com/protocolbuffers/protobuf.git) registered for path 'third_party/protobuf'
Cloning into '/content/TensorRT/parsers/onnx'...
Cloning into '/content/TensorRT/third_party/cub'...
Cloning into '/content/TensorRT/third_party/protobuf'...
Submodule path 'parsers/onnx': checked out '078f20b8fbcbb802aa5ffcd2134519721a61b88c'
Submodule 'third_party/onnx' (https://github.com/onnx/onnx.git) reg

Voila..TensorRT components will be inside TensorRT/build. I for one wanted onnx2trt executable. GG!